<a href="https://colab.research.google.com/github/frankuc19/fran/blob/main/examples/notebook/constraint_solver/vrp_time_windows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


# vrp_time_windows

<table align="left">
<td>
<a href="https://colab.research.google.com/github/google/or-tools/blob/main/examples/notebook/constraint_solver/vrp_time_windows.ipynb"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/colab_32px.png"/>Run in Google Colab</a>
</td>
<td>
<a href="https://github.com/google/or-tools/blob/main/ortools/constraint_solver/samples/vrp_time_windows.py"><img src="https://raw.githubusercontent.com/google/or-tools/main/tools/github_32px.png"/>View source on GitHub</a>
</td>
</table>

First, you must install [ortools](https://pypi.org/project/ortools/) package in this colab.

In [1]:
%pip install ortools
!pip install h3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.4 MB/s eta 0:00:00



Vehicles Routing Problem (VRP) with Time Windows.


In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp



def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix"] = [
        [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
        [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
        [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
        [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
        [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
        [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
        [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
        [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
        [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
        [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
        [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
        [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
        [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
        [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
        [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
        [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
        [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
    ]
    data["time_windows"] = [
        (0, 5),  # depot
        (7, 12),  # 1
        (10, 15),  # 2
        (16, 18),  # 3
        (10, 13),  # 4
        (0, 5),  # 5
        (5, 10),  # 6
        (0, 4),  # 7
        (5, 10),  # 8
        (0, 3),  # 9
        (10, 16),  # 10
        (10, 15),  # 11
        (0, 5),  # 12
        (5, 10),  # 13
        (7, 8),  # 14
        (10, 15),  # 15
        (11, 15),  # 16
    ]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")


def main():
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        30,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data["time_windows"]):
        if location_idx == data["depot"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data["depot"]
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data["time_windows"][depot_idx][0], data["time_windows"][depot_idx][1]
        )

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


main()



In [8]:
# ----------------------------------------------------------
# Parte 1: Imports y Funciones del Script de Asignación
# ----------------------------------------------------------
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import math
import h3 # Asegúrate de que h3 está instalado (!pip install h3)
from tqdm import tqdm # Para la barra de progreso
import os # Para verificar existencia de archivos
# --- Importar para descarga en Colab ---
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
# -------------------------------------

# (El resto de imports de OR-Tools están más abajo)

# ----------------------------------------------------------
# Parte 2: Constantes y Configuraciones Globales
# ----------------------------------------------------------
RADIO_TIERRA_KM = 6371
PRECISION_H3 = 3
DEFAULT_AVG_SPEED_KMH = 40

# Parámetros del Modelo (¡Ajusta según sea necesario!)
PARAM_MAX_MOVILES = 100
PARAM_MAX_RESERVAS_POR_MOVIL = 5
PARAM_MAX_HORAS_POR_MOVIL = 10
ID_COLUMN_NAME = 'job_id' # <--- ¡¡¡ VERIFICA ESTE NOMBRE !!!
SOLVER_TIME_LIMIT_SECONDS = 60
MAX_SLACK_MINUTES = 30
PICKUP_WINDOW_DURATION_MINUTES = 60

# Valor grande para representar infinito (en minutos)
INF_TIME = 30 * 24 * 60

# Variables globales para datos precalculados
summary_df = pd.DataFrame()
avg_speed_kmh = DEFAULT_AVG_SPEED_KMH

# ----------------------------------------------------------
# Parte 3: Funciones de Cálculo (haversine, simulate_h3_str, ...)
# (Se mantienen igual que en la versión anterior - omitidas aquí por brevedad)
# ... (incluye aquí las funciones haversine, simulate_h3_str,
#      precompute_historical_averages, get_travel_time_minutes) ...
# COPIAR FUNCIONES DE CÁLCULO DESDE LA RESPUESTA ANTERIOR AQUÍ
def haversine(lat1, lon1, lat2, lon2):
    """Calcula la distancia Haversine entre dos puntos en KM."""
    coords = [lat1, lon1, lat2, lon2]
    if any(map(lambda x: pd.isna(x) or x is None, coords)):
        return np.nan
    try:
        lat1, lon1, lat2, lon2 = map(np.radians, coords)
        dlat, dlon = lat2 - lat1, lon2 - lon1
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        distance = RADIO_TIERRA_KM * c
        return distance
    except ValueError as e:
        # print(f"Error calculando haversine para {coords}: {e}") # Descomentar para debug
        return np.nan

def simulate_h3_str(lat, lon, precision=PRECISION_H3):
     """Genera un ID H3 string simulado basado en lat/lon redondeados."""
     if pd.isna(lat) or pd.isna(lon):
         return None
     return f"{round(lat, precision)}_{round(lon, precision)}"

def precompute_historical_averages(df_hist):
    """Calcula tiempos promedio desde datos históricos y velocidad fallback."""
    global summary_df, avg_speed_kmh
    required_hist_cols = {'latrecogida', 'lonrecogida', 'latdestino', 'londestino', 'tiempoestimada'}
    if not required_hist_cols.issubset(df_hist.columns):
        missing = required_hist_cols - set(df_hist.columns)
        raise ValueError(f"El DataFrame histórico no tiene las columnas necesarias: {missing}")
    print("Limpiando datos históricos...")
    initial_rows = len(df_hist)
    df_hist = df_hist.dropna(subset=list(required_hist_cols))
    df_hist['tiempoestimada'] = pd.to_numeric(df_hist['tiempoestimada'], errors='coerce')
    df_hist = df_hist.dropna(subset=['tiempoestimada'])
    df_hist = df_hist[df_hist['tiempoestimada'] > 0]
    cleaned_rows_1 = len(df_hist)
    print(f"  {initial_rows - cleaned_rows_1} filas eliminadas por NaNs o tiempo inválido.")
    if df_hist.empty:
        # ... (manejo de error igual que antes) ...
        print("Advertencia: No hay datos históricos válidos después de la limpieza inicial.")
        summary_df = pd.DataFrame(columns=['h3_origin', 'h3_destino', 'avg_travel_time_min'])
        avg_speed_kmh = DEFAULT_AVG_SPEED_KMH
        print(f"Usando velocidad fallback por defecto: {avg_speed_kmh} km/h")
        return

    print("Calculando H3 y distancias para datos históricos...")
    df_hist['h3_origin'] = df_hist.apply(lambda row: simulate_h3_str(row['latrecogida'], row['lonrecogida']), axis=1)
    df_hist['h3_destino'] = df_hist.apply(lambda row: simulate_h3_str(row['latdestino'], row['londestino']), axis=1)
    df_hist['distance_km'] = df_hist.apply(lambda row: haversine(row['latrecogida'], row['lonrecogida'], row['latdestino'], row['londestino']), axis=1)
    df_hist = df_hist.dropna(subset=['h3_origin', 'h3_destino', 'distance_km'])
    df_hist = df_hist[df_hist['distance_km'] > 0.01]
    cleaned_rows_2 = len(df_hist)
    print(f"  {cleaned_rows_1 - cleaned_rows_2} filas eliminadas por cálculo inválido de H3/distancia.")
    if df_hist.empty:
        # ... (manejo de error igual que antes) ...
        print("Advertencia: No hay datos históricos válidos después de calcular H3/distancia.")
        summary_df = pd.DataFrame(columns=['h3_origin', 'h3_destino', 'avg_travel_time_min'])
        avg_speed_kmh = DEFAULT_AVG_SPEED_KMH
        print(f"Usando velocidad fallback por defecto: {avg_speed_kmh} km/h")
        return

    print("Calculando tiempos promedio agrupados por H3...")
    avg_times_df = df_hist.groupby(['h3_origin', 'h3_destino'], as_index=False)['tiempoestimada'].median()
    avg_times_df.rename(columns={'tiempoestimada': 'avg_travel_time_min'}, inplace=True)
    summary_df = avg_times_df
    print(f"Se calcularon promedios históricos (mediana). {len(summary_df)} pares H3 origen-destino encontrados.")

    print("Calculando velocidad promedio global (fallback)...")
    df_hist['speed_kmh'] = (df_hist['distance_km'] / df_hist['tiempoestimada']) * 60
    df_hist_valid_speed = df_hist[(df_hist['speed_kmh'] > 1) & (df_hist['speed_kmh'] < 120)]
    if not df_hist_valid_speed.empty:
        avg_speed_kmh = df_hist_valid_speed['speed_kmh'].median()
        print(f"Velocidad promedio (mediana) calculada (fallback): {avg_speed_kmh:.2f} km/h")
    else:
        avg_speed_kmh = DEFAULT_AVG_SPEED_KMH
        print(f"No se pudieron calcular velocidades válidas. Usando fallback por defecto: {avg_speed_kmh} km/h")

def get_travel_time_minutes(h3_origin, h3_dest, fallback_distance_km):
    """Obtiene el tiempo de viaje promedio en minutos entre H3. Usa fallback si no se encuentra."""
    if h3_origin is not None and h3_dest is not None and summary_df is not None and not summary_df.empty:
        row = summary_df[(summary_df['h3_origin'] == h3_origin) & (summary_df['h3_destino'] == h3_dest)]
        if not row.empty:
            avg_time = row['avg_travel_time_min'].iloc[0]
            if pd.notna(avg_time):
                 return int(round(avg_time))
    if pd.notna(fallback_distance_km) and avg_speed_kmh > 0:
        calculated_time = (fallback_distance_km / avg_speed_kmh) * 60
        return int(round(calculated_time))
    return 60
# ----------------------------------------------------------


# ----------------------------------------------------------
# Parte 4: Funciones de OR-Tools y Ejecución Principal
# ----------------------------------------------------------
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# --- MODIFICACIÓN: print_solution ahora devuelve datos ---
def process_and_print_solution(num_jobs, tasks, data, manager, routing, solution):
    """Procesa la solución, la imprime y devuelve datos estructurados."""
    global ID_COLUMN_NAME

    print(f"\nObjective (Total Time): {solution.ObjectiveValue()} min")
    time_dimension = routing.GetDimensionOrDie("Time")
    jobs_dimension = routing.GetDimensionOrDie("Jobs") # Puede ser None

    solution_routes = [] # Lista para almacenar la información de cada ruta usada
    assigned_job_node_indices = set() # Para rastrear qué trabajos se asignaron

    print("\n--- Routes ---")
    for vehicle_id in range(data["num_vehicles"]):
        if not routing.IsVehicleUsed(solution, vehicle_id):
            continue

        route_nodes_indices = []
        route_details_list = [] # Lista de diccionarios para los detalles de esta ruta
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        plan_output += "  Start (Source) -> "
        route_time = 0

        while True:
            node_index_internal = index
            node_index_mapped = manager.IndexToNode(node_index_internal)

            if 0 <= node_index_mapped < num_jobs: # Es un nodo de trabajo
                job_idx = node_index_mapped
                task_info = tasks[job_idx]
                time_var = time_dimension.CumulVar(node_index_internal)
                arrival_time_min = solution.Min(time_var)
                arrival_time_max = solution.Max(time_var)
                job_count_at_node = "N/A"
                if jobs_dimension:
                    try:
                        jobs_var = jobs_dimension.CumulVar(node_index_internal)
                        job_count_at_node = solution.Value(jobs_var)
                    except Exception: pass

                plan_output += (
                    f"Job_{task_info.get(ID_COLUMN_NAME, job_idx)}"
                    f" [P:{task_info['pickup_time_str']}]"
                    f" (Arr: {arrival_time_min}-{arrival_time_max} min)"
                    f" (Jobs: {job_count_at_node})"
                    " -> "
                )
                # Guardar detalles del trabajo asignado
                job_detail = {
                    "or_tools_vehicle_id": vehicle_id, # ID interno de OR-Tools
                    "job_id_unique": task_info.get(ID_COLUMN_NAME, job_idx), # El ID del trabajo
                    "node_index": job_idx, # Índice 0..num_jobs-1
                    "pickup_time_original": task_info['pickup_time_str'],
                    "arrival_minutes_min": arrival_time_min,
                    "arrival_minutes_max": arrival_time_max,
                    "job_count_at_node": job_count_at_node,
                    # Añadir otros datos relevantes de task_info si es necesario
                    "estimated_payment": task_info.get('estimated_payment', 0), # Incluir pago
                    "pickup_lat": task_info.get('pickup_lat'),
                    "pickup_lon": task_info.get('pickup_lon'),
                    "dropoff_lat": task_info.get('dropoff_lat'),
                    "dropoff_lon": task_info.get('dropoff_lon'),
                }
                route_details_list.append(job_detail)
                route_nodes_indices.append(job_idx)
                assigned_job_node_indices.add(job_idx) # Marcar como asignado

            if routing.IsEnd(index):
                if node_index_mapped == data["sink_node"]:
                    time_var_end = time_dimension.CumulVar(node_index_internal)
                    route_time = solution.Min(time_var_end)
                    plan_output += f"End (Sink) [Total Time: {route_time} min]"
                else:
                    plan_output += f"End (Node {node_index_mapped}?) [Incomplete Route Trace]"
                    route_time = solution.ObjectiveValue()
                break
            index = solution.Value(routing.NextVar(index))

        print(plan_output)
        print(f"  Jobs in route: {len(route_nodes_indices)}")

        # Guardar información de la ruta completa
        solution_routes.append({
            "or_tools_vehicle_id": vehicle_id,
            "route_nodes": route_nodes_indices, # Lista de índices de trabajos
            "route_details": route_details_list, # Lista de detalles por trabajo
            "total_route_time_min": route_time,
            "total_jobs_in_route": len(route_nodes_indices)
        })

    total_jobs_assigned = len(assigned_job_node_indices)
    print(f"\n--- Summary ---")
    print(f"Total time of all routes: {solution.ObjectiveValue()} min") # Usar valor objetivo para tiempo total
    print(f"Total jobs assigned: {total_jobs_assigned} / {num_jobs}")

    # Devolver los datos procesados
    return solution_routes, assigned_job_node_indices, total_jobs_assigned
# --- Fin de la función modificada ---


def main_ortools(df_hist_path, df_pred_path):
    """Función principal que carga datos, configura, resuelve y exporta."""
    global ID_COLUMN_NAME

    # 1. Cargar Datos y 2. Validar Predicciones (igual que antes)
    print("Cargando archivos...")
    try:
        df_hist = pd.read_csv(df_hist_path)
        dtypes_pred = {'latrecogida': float, 'lonrecogida': float, 'latdestino': float, 'londestino': float}
        df_pred = pd.read_csv(df_pred_path, dtype=dtypes_pred, low_memory=False)
        print(f"Históricos cargados: {len(df_hist)} filas")
        print(f"Predicciones cargadas: {len(df_pred)} filas")
    except FileNotFoundError as e: print(f"Error: No se encontró el archivo {e.filename}"); return
    except Exception as e: print(f"Error cargando archivos CSV: {e}"); return

    print("Validando y preparando datos de predicción...")
    required_pred_cols = {'latrecogida', 'lonrecogida', 'latdestino', 'londestino', 'pickup_datetime', ID_COLUMN_NAME}
    if not required_pred_cols.issubset(df_pred.columns):
         missing_cols = required_pred_cols - set(df_pred.columns)
         print(f"Error: Faltan columnas requeridas en predicciones: {missing_cols}")
         print(f"Columnas encontradas: {list(df_pred.columns)}"); return
    if 'estimated_payment' not in df_pred.columns:
        print("Advertencia: Columna 'estimated_payment' no encontrada. Se usará 0.")
        df_pred['estimated_payment'] = 0
    else:
         # Asegurarse que sea numérico
         df_pred['estimated_payment'] = pd.to_numeric(df_pred['estimated_payment'], errors='coerce').fillna(0)


    try:
        df_pred['HoraFecha'] = pd.to_datetime(df_pred['pickup_datetime'], errors='coerce')
    except Exception as e: print(f"Error convirtiendo 'pickup_datetime' a fecha: {e}"); return
    df_pred = df_pred.dropna(subset=['HoraFecha'])
    if df_pred.empty: print("Error: No hay fechas válidas ('pickup_datetime')."); return
    start_horizon_time = df_pred['HoraFecha'].min()
    print(f"Horizonte de planificación inicia en: {start_horizon_time}")

    # 3. Precomputar Promedios (igual que antes)
    print("Precalculando promedios históricos...")
    try: precompute_historical_averages(df_hist)
    except ValueError as e: print(f"Error en precomputo histórico: {e}"); return
    except Exception as e: print(f"Error inesperado en precomputo histórico: {e}"); return

    # 4. Preparar Lista de Tareas (igual que antes, asegurando 'estimated_payment')
    print("Preparando lista de tareas (trabajos)...")
    tasks = []
    job_durations_min = []
    pickup_coords = []
    dropoff_coords = []
    pickup_windows_min = []
    skipped_jobs = 0
    for idx, row in df_pred.iterrows():
        required_row_data = ['latrecogida', 'lonrecogida', 'latdestino', 'londestino', 'HoraFecha']
        if any(pd.isna(row[col]) for col in required_row_data):
             job_id_display = row.get(ID_COLUMN_NAME, f"index_{idx}")
             print(f"Advertencia: Saltando trabajo '{job_id_display}' por datos faltantes.")
             skipped_jobs += 1; continue
        h3_pickup = simulate_h3_str(row['latrecogida'], row['lonrecogida'])
        h3_dropoff = simulate_h3_str(row['latdestino'], row['londestino'])
        distance_job_km = haversine(row['latrecogida'], row['lonrecogida'], row['latdestino'], row['londestino'])
        job_duration = get_travel_time_minutes(h3_pickup, h3_dropoff, distance_job_km)
        job_durations_min.append(job_duration)
        earliest_pickup_time = row['HoraFecha']
        pickup_start_min = int(round((earliest_pickup_time - start_horizon_time).total_seconds() / 60))
        pickup_end_min = pickup_start_min + PICKUP_WINDOW_DURATION_MINUTES
        pickup_windows_min.append((max(0, pickup_start_min), pickup_end_min))
        task_data = {
            "original_index": idx,
            ID_COLUMN_NAME : row.get(ID_COLUMN_NAME),
            "pickup_lat": row['latrecogida'],"pickup_lon": row['lonrecogida'],
            "dropoff_lat": row['latdestino'],"dropoff_lon": row['londestino'],
            "h3_pickup": h3_pickup,"h3_dropoff": h3_dropoff,
            "pickup_time": earliest_pickup_time,"pickup_time_str": earliest_pickup_time.strftime('%Y-%m-%d %H:%M'),
            # Incluir estimated_payment en los datos de la tarea
            "estimated_payment": row.get('estimated_payment', 0)
        }
        tasks.append(task_data)
        pickup_coords.append((row['latrecogida'], row['lonrecogida']))
        dropoff_coords.append((row['latdestino'], row['londestino']))
    num_jobs = len(tasks)
    if num_jobs == 0: print(f"Error: No hay trabajos válidos (se saltaron {skipped_jobs})."); return
    print(f"Número de trabajos a asignar: {num_jobs} (se saltaron {skipped_jobs})")

    # 5. Configurar Nodos y Vehículos (igual que antes)
    num_vehicles = min(PARAM_MAX_MOVILES, num_jobs)
    print(f"Número de vehículos a usar: {num_vehicles}")
    num_locations_total = num_jobs + 2
    source_node = num_jobs
    sink_node = num_jobs + 1

    # 6. Precalcular Matriz de Tiempos (igual que antes, con tqdm)
    print("Precalculando matriz de tiempos de viaje inter-trabajos...")
    travel_times_min = [[0] * num_jobs for _ in range(num_jobs)]
    for i in tqdm(range(num_jobs), desc="Calculando Matriz Tiempos", unit="job"):
        for j in range(num_jobs):
            if i == j: travel_times_min[i][j] = INF_TIME; continue
            lat1, lon1 = dropoff_coords[i]; lat2, lon2 = pickup_coords[j]
            h3_origin = tasks[i]["h3_dropoff"]; h3_dest = tasks[j]["h3_pickup"]
            distance_km_fallback = haversine(lat1, lon1, lat2, lon2)
            travel_times_min[i][j] = get_travel_time_minutes(h3_origin, h3_dest, distance_km_fallback)
    print("Matriz de tiempos inter-trabajos calculada.")

    # 7. Crear Manager y Modelo (igual que antes)
    print("Configurando modelo OR-Tools...");
    try:
        manager = pywrapcp.RoutingIndexManager(num_locations_total, num_vehicles, [source_node] * num_vehicles, [sink_node] * num_vehicles)
        routing = pywrapcp.RoutingModel(manager); print("Manager y Modelo creados.")
    except Exception as e: print(f"Error creando modelo OR-Tools: {e}"); return

    # 8. Callback de Tránsito (igual que antes)
    def time_callback(from_index, to_index):
        try:
            from_node = manager.IndexToNode(from_index); to_node = manager.IndexToNode(to_index)
            inter_travel_time = 0
            if 0 <= from_node < num_jobs and 0 <= to_node < num_jobs: inter_travel_time = travel_times_min[from_node][to_node]
            elif from_node == source_node and 0 <= to_node < num_jobs: inter_travel_time = 0
            elif 0 <= from_node < num_jobs and to_node == sink_node: inter_travel_time = 0
            elif from_node == source_node and to_node == sink_node: inter_travel_time = 0
            else: return INF_TIME
            origin_job_duration = 0
            if 0 <= from_node < num_jobs: origin_job_duration = job_durations_min[from_node]
            total_time = int(round(inter_travel_time + origin_job_duration))
            return min(total_time, INF_TIME)
        except IndexError: return INF_TIME
        except Exception: return INF_TIME
    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    print("Callback de tiempo registrado y costo de arcos establecido.")

    # 9. Dimensión de Tiempo (igual que antes, con debug de SetRange)
    print("Añadiendo dimensión de Tiempo..."); time_dimension_name = "Time"
    max_route_time_min = PARAM_MAX_HORAS_POR_MOVIL * 60
    max_window_end = max(w[1] for w in pickup_windows_min) if pickup_windows_min else 0
    horizon = max(max_window_end + max(job_durations_min) if job_durations_min else 0, max_route_time_min); horizon += max_route_time_min
    print(f"  Horizonte: {horizon} min, Capacidad Ruta: {max_route_time_min} min, Slack: {MAX_SLACK_MINUTES} min")
    routing.AddDimension(transit_callback_index, MAX_SLACK_MINUTES, max_route_time_min, False, time_dimension_name)
    time_dimension = routing.GetDimensionOrDie(time_dimension_name); print(f"Dimensión '{time_dimension_name}' añadida.")
    print("Aplicando ventanas de tiempo a los trabajos...");
    try:
        for job_idx in range(num_jobs):
            index = manager.NodeToIndex(job_idx); start_win, end_win = pickup_windows_min[job_idx]
            job_id_display = tasks[job_idx].get(ID_COLUMN_NAME, job_idx)
            # print(f"DEBUG: SetRange Job {job_id_display} [{start_win}, {end_win}]") # Descomentar para debug detallado
            if start_win > end_win: raise ValueError(f"Ventana inválida para Job {job_id_display}: [{start_win}, {end_win}]")
            time_dimension.CumulVar(index).SetRange(start_win, end_win)
        source_index = manager.NodeToIndex(source_node)
        # print(f"DEBUG: SetRange Source [0, {horizon}]") # Descomentar para debug detallado
        time_dimension.CumulVar(source_index).SetRange(0, horizon)
        print("Ventanas de tiempo aplicadas.")
    except Exception as e:
        print(f"\n!!! ERROR CRÍTICO al aplicar ventana de tiempo para Job {job_id_display} [{start_win}, {end_win}]: {e} !!!"); return

    # 10. Dimensión de Conteo de Trabajos (igual que antes)
    print(f"Añadiendo dimensión de Conteo de Trabajos (max: {PARAM_MAX_RESERVAS_POR_MOVIL})...")
    jobs_dimension_name = "Jobs";
    try:
        def demand_callback(from_index):
            from_node = manager.IndexToNode(from_index); return 1 if 0 <= from_node < num_jobs else 0
        demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
        routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, [PARAM_MAX_RESERVAS_POR_MOVIL] * num_vehicles, True, jobs_dimension_name)
        print(f"Dimensión '{jobs_dimension_name}' añadida.")
    except Exception as e: print(f"Error añadiendo dimensión '{jobs_dimension_name}': {e}")

    # 11. Configurar Búsqueda y Resolver (igual que antes)
    print("Configurando parámetros de búsqueda y resolviendo...")
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    search_parameters.time_limit.seconds = SOLVER_TIME_LIMIT_SECONDS
    search_parameters.log_search = True
    print(f"Límite de tiempo del solver: {SOLVER_TIME_LIMIT_SECONDS} segundos.")
    solution = routing.SolveWithParameters(search_parameters)

    # 12. Procesar Solución, Exportar y Mostrar Resumen
    if solution:
        print("\n********************* Solución Encontrada! *********************")
        # --- Procesar la solución para obtener datos estructurados ---
        solution_routes, assigned_job_node_indices, total_jobs_assigned = process_and_print_solution(
            num_jobs, tasks, {"num_vehicles": num_vehicles, "source_node": source_node, "sink_node": sink_node},
            manager, routing, solution
        )

        # --- Crear lista de trabajos asignados para df_rutas ---
        rutas_asignadas_list = []
        vehicle_map = {} # Mapea ID interno de OR-Tools a ID secuencial 1..N
        next_movil_id = 1
        for route in solution_routes:
            or_tools_vehicle_id = route['or_tools_vehicle_id']
            if or_tools_vehicle_id not in vehicle_map:
                vehicle_map[or_tools_vehicle_id] = next_movil_id
                current_movil_id = next_movil_id
                next_movil_id += 1
            else:
                current_movil_id = vehicle_map[or_tools_vehicle_id]

            # Añadir cada detalle de trabajo con el movil_id correcto
            for job_detail in route['route_details']:
                job_detail_export = job_detail.copy()
                job_detail_export['movil_id'] = current_movil_id
                # Eliminar claves internas si no se desean en el CSV
                # del job_detail_export['or_tools_vehicle_id']
                # del job_detail_export['node_index']
                rutas_asignadas_list.append(job_detail_export)

        df_rutas = pd.DataFrame(rutas_asignadas_list)
        print(f"\nDataFrame 'df_rutas' creado con {len(df_rutas)} filas.")

        # --- Crear lista de trabajos NO asignados ---
        reservas_no_asignadas_list = []
        for job_idx in range(num_jobs):
            if job_idx not in assigned_job_node_indices:
                unassigned_task = tasks[job_idx].copy()
                # Añadir motivo (simplificado para OR-Tools)
                unassigned_task['motivo_no_asignado'] = "No incluido en solución OR-Tools"
                reservas_no_asignadas_list.append(unassigned_task)

        df_no_asignadas = pd.DataFrame(reservas_no_asignadas_list)
        print(f"DataFrame 'df_no_asignadas' creado con {len(df_no_asignadas)} filas.")

        # --- Exportar a CSV ---
        rutas_csv_path = "rutas_asignadas_ortools.csv"
        no_asignadas_csv_path = "reservas_no_asignadas_ortools.csv"
        try:
            df_rutas.to_csv(rutas_csv_path, index=False)
            print(f"Resultados de rutas asignadas exportados a: {rutas_csv_path}")
            df_no_asignadas.to_csv(no_asignadas_csv_path, index=False)
            print(f"Resultados de reservas no asignadas exportados a: {no_asignadas_csv_path}")

            # --- Descargar en Google Colab (si aplica) ---
            if IN_COLAB:
                print("Iniciando descarga de archivos en Colab...")
                files.download(rutas_csv_path)
                files.download(no_asignadas_csv_path)
            # --------------------------------------------

        except Exception as e:
            print(f"\nError al exportar archivos CSV: {e}")

        # --- Imprimir Resumen Final ---
        print("\n--- Resumen Final ---")
        num_vehicles_used = len(solution_routes) # Número de vehículos con ruta asignada
        total_payment_assigned = df_rutas['estimated_payment'].sum() if 'estimated_payment' in df_rutas.columns else 0

        print(f"✅ Móviles usados: {num_vehicles_used}")
        print(f"✅ Trabajos asignados: {total_jobs_assigned}")
        print(f"🚫 Trabajos NO asignados: {len(df_no_asignadas)}")
        # Formatear el pago total como moneda
        try:
             print(f"💰 Ganancia total estimada (asignados): ${total_payment_assigned:,.0f}")
        except ValueError:
             print(f"💰 Ganancia total estimada (asignados): {total_payment_assigned}") # Sin formato si falla

        print("**********************************************************\n")

    else:
        # --- Si no se encontró solución ---
        print("\n******************** No se encontró solución ********************")
        print("  No se generaron archivos de resultados.")
        print("  Posibles causas: Restricciones muy estrictas, datos inconsistentes,")
        print("                   límite de tiempo insuficiente, no hay suficientes vehículos.")
        print("**********************************************************\n")


# ----------------------------------------------------------
# Parte 5: Bloque de Ejecución Principal
# ----------------------------------------------------------
if __name__ == "__main__":
      # --- CONFIGURAR RUTAS A ARCHIVOS ---
      hist_file = '/content/distancias H3 2.0 (Hist).csv'        # ¡¡¡ CAMBIA ESTO !!!
      pred_file = '/content/distancias H3 2.0 (Pred) - 25-04 05-15.csv'    # ¡¡¡ CAMBIA ESTO !!!
      # ------------------------------------

      print(f"Iniciando proceso de optimización de rutas...")
      print(f"Archivo histórico: {hist_file}")
      print(f"Archivo de predicciones: {pred_file}")
      print(f"ID Column Name expected in predictions: '{ID_COLUMN_NAME}'")

      if not os.path.exists(hist_file):
          print(f"\nERROR FATAL: El archivo histórico '{hist_file}' no existe.")
      elif not os.path.exists(pred_file):
          print(f"\nERROR FATAL: El archivo de predicciones '{pred_file}' no existe.")
      else:
          main_ortools(hist_file, pred_file)

      print("Proceso finalizado.")

Iniciando proceso de optimización de rutas...
Archivo histórico: /content/distancias H3 2.0 (Hist).csv
Archivo de predicciones: /content/distancias H3 2.0 (Pred) - 25-04 05-15.csv
ID Column Name expected in predictions: 'job_id'
Cargando archivos...
Históricos cargados: 461962 filas
Predicciones cargadas: 215 filas
Validando y preparando datos de predicción...
Horizonte de planificación inicia en: 2025-04-25 05:00:00
Precalculando promedios históricos...
Limpiando datos históricos...
  85796 filas eliminadas por NaNs o tiempo inválido.
Calculando H3 y distancias para datos históricos...


<ipython-input-8-384875a72c93>:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hist['tiempoestimada'] = pd.to_numeric(df_hist['tiempoestimada'], errors='coerce')


  7 filas eliminadas por cálculo inválido de H3/distancia.
Calculando tiempos promedio agrupados por H3...
Se calcularon promedios históricos (mediana). 79205 pares H3 origen-destino encontrados.
Calculando velocidad promedio global (fallback)...
Velocidad promedio (mediana) calculada (fallback): 36.10 km/h
Preparando lista de tareas (trabajos)...
Número de trabajos a asignar: 215 (se saltaron 0)
Número de vehículos a usar: 100
Precalculando matriz de tiempos de viaje inter-trabajos...


Calculando Matriz Tiempos: 100%|██████████| 215/215 [17:24<00:00,  4.86s/job]

Matriz de tiempos inter-trabajos calculada.
Configurando modelo OR-Tools...
Manager y Modelo creados.
Callback de tiempo registrado y costo de arcos establecido.
Añadiendo dimensión de Tiempo...
  Horizonte: 1537 min, Capacidad Ruta: 600 min, Slack: 30 min
Dimensión 'Time' añadida.
Aplicando ventanas de tiempo a los trabajos...

!!! ERROR CRÍTICO al aplicar ventana de tiempo para Job 23673492 [620, 680]: CP Solver fail !!!
Proceso finalizado.
